In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time 
import random

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Rob\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [3]:
# url_list = ['https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784109/NEWCAR_FEATURED',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784118/NEWCAR_FEATURED',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=78704&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d2412#listing=333651233/NONE',
#            'https://www.cargurus.com/Cars/new/searchresults.action?zip=76012&showNegotiable=true&sortDir=ASC&sourceContext=homePageNewCarTab_false_0&distance=50&sortType=PRICE&entitySelectingHelper.selectedEntity=d320#listing=333784109/NEWCAR_FEATURED']

url_df = pd.read_csv('../Resources/all_urls_list.csv')
url_list = url_df['urls']
url_list

0       https://www.cargurus.com/Cars/new/searchresult...
1       https://www.cargurus.com/Cars/new/searchresult...
2       https://www.cargurus.com/Cars/new/searchresult...
3       https://www.cargurus.com/Cars/new/searchresult...
4       https://www.cargurus.com/Cars/new/searchresult...
                              ...                        
2216    https://www.cargurus.com/Cars/new/searchresult...
2217    https://www.cargurus.com/Cars/new/searchresult...
2218    https://www.cargurus.com/Cars/new/searchresult...
2219    https://www.cargurus.com/Cars/new/searchresult...
2220    https://www.cargurus.com/Cars/new/searchresult...
Name: urls, Length: 2221, dtype: object

In [5]:
cars_df = pd.DataFrame({})


In [6]:
browser.visit(url_list[0])

In [7]:
failed_tries = []

In [9]:
start = 200
end = 400
# start with just 1 loop
for i in range(start,end):
    browser.visit(url_list[i])
    time.sleep(random.randint(3,6))
    
    # place into try / except block:
    try:
        print(f"Adding {i} of {end}")
        # scrape using find by tag (section).  Should return list.
        keys = browser.find_by_tag('dl').find_by_tag('dt')
        values = browser.find_by_tag('dl').find_by_tag('dd')
        print('.', end='')
        # loop through section list and find the one that has "Dealer's Price"
#         section_list = []
#         for e in elems:
#             section_list.append(e.html)
        # extract section list item into section variable

        # run replaces on tags in section to turn it into table
        

        # put table into single dataframe temp_df (NOT scrape_cars_df)
        temp_df = pd.DataFrame({})
        for j in range(len(keys)):
            if values[j].text == 'Contact the dealer for pricing details':
                values.pop(j)
            temp_df[keys[j].text]= [values[j].text]
        temp_df
        print('.', end='')
        # create search string to find remaining car data
        # search_str = " - " + temp_df["Dealer's Price:"][0]
        # find <h1> tag which contains search_str
        search_str = "IpF2YF"
        title = browser.find_by_tag('h1')
        price = title[0].html.split(' ')[-1]
        model = title[0].html.split(' ')[2]
        
        temp_df['Price:'] = [price]
        temp_df['Model:'] = [model]
        print('.', end='')
        # split element string by space
        # year = e[0]
        # make = e[1]
        # model = e[2]
        dealernames = browser.find_by_tag('a')
        for d in dealernames:
            if d['data-cg-ft'] == 'vdp-dealer-cg-homepage-link':
                dealername = d.text
        temp_df['Dealer Name:'] = [dealername]
        # add columns and data to dataframe (temp_df['year'] = [year])
        print('.', end='')
        
        dealeraddress = browser.find_by_tag('span')
        for d in dealeraddress:
            if d['data-track-ui'] == 'dealer-address':
                dealeradd = d.text
        dealeradd = dealeradd.replace('\n', '<br>')        
        temp_df["Dealer Address:"] = [dealeradd]
        # append new dataframe to scrape_cars_df if successful
        print('.', end='')
        
        cars_df = pd.concat([cars_df, temp_df], axis=0, ignore_index=True)
        print(' Success!')
        
    
    # except: add i into failed list
    except:
        failed_tries.append(i)
        print(' Error found.')
cars_df.to_csv(f'../Resources/{start}-{end}.csv')

Adding 200 of 400
..... Success!
Adding 201 of 400
..... Success!


KeyboardInterrupt: 

In [11]:
dealeraddress = browser.find_by_tag('span')
for d in dealeraddress:
    if d['data-track-ui'] == 'dealer-address':
        dealeradd = d.text
dealeradd = dealeradd.replace('\n', '<br>')  
dealeradd
#         temp_df["Dealer Address:"] = [dealeradd]

'2536 Forest Ln Dallas, TX 75234'

In [22]:
keys = browser.find_by_tag('dl').find_by_tag('dt')
values = browser.find_by_tag('dl').find_by_tag('dd')
temp_df = pd.DataFrame({})


for j in range(len(keys)):
    if values[j].text == 'Contact the dealer for pricing details':
        values.pop(j)
    temp_df[keys[j].text]= [values[j].text]
temp_df

# values[2].text

,Dealer's Price:,MSRP:,Location:,Mileage:,Transmission:,Exterior Color:,Interior Color:,Est. Gas Mileage*:,Engine:,Drivetrain:,Fuel Type:,VIN:,Stock #:,Condition:
0,"$51,790","$51,790","Dallas, TX\n18 mi away",---,8-Speed Automatic,Black Clearcoat,Black,22 MPG City\n29 MPG Highway\n25 MPG Combined,260 hp 3L V6 Diesel,Four-Wheel Drive,Diesel,1C4HJXDMXNW242806,F0825,New


In [12]:
cars_df

,Dealer's Price:,MSRP:,Location:,Mileage:,Transmission:,Exterior Color:,Interior Color:,Est. Gas Mileage*:,Engine:,Drivetrain:,Fuel Type:,VIN:,Stock #:,Condition:,Price:,Model:,Dealer Name:,Dealer Address:,Delivery:,Price + Delivery:
0,"$41,305","$40,305",Contact the dealer for pricing details,"Fairfield, TX\n92 mi away",2 miles,8-Speed Automatic,Black Clearcoat,Black,22 MPG City\n24 MPG Highway\n22 MPG Combined,270 hp 2L I4,Four-Wheel Drive,Gasoline,1C4GJXAN8NW223826,223826,"$41,305",Wrangler,Freestone Autoplex,"1201 Hwy 84 West Fairfield, TX 75840",NaN,NaN
1,"$41,305","$40,305",Contact the dealer for pricing details,"Fairfield, TX\n92 mi away",2 miles,8-Speed Automatic,Firecracker Red Clearcoat,Black,22 MPG City\n24 MPG Highway\n22 MPG Combined,270 hp 2L I4,Four-Wheel Drive,Gasoline,1C4GJXAN1NW223828,223828,"$41,442",Wrangler,Freestone Autoplex,"1201 Hwy 84 West Fairfield, TX 75840",$136,"$41,442"
2,"$42,065","$42,065",Contact the dealer for pricing details,"Richardson, TX\n28 mi away",0 miles,8-Speed Automatic,Bright White,Black,22 MPG City\n24 MPG Highway\n22 MPG Combined,270 hp 2L I4,Four-Wheel Drive,Gasoline,1C4GJXAN2NW186076,186076,"$42,065",Wrangler,Richardson Chrysler Jeep Dodge Ram,"1321 N Central Expy Richardson, TX 75080",NaN,NaN
3,"$42,555","$42,555",Contact the dealer for pricing details,"Fort Worth, TX\n19 mi away",4 miles,8-Speed Automatic,Sarge Green Clearcoat,Black,18 MPG City\n23 MPG Highway\n20 MPG Combined,285 hp 3.6L V6,Four-Wheel Drive,Gasoline,1C4GJXAG3NW268061,268061,"$42,555",Wrangler,Moritz Chrysler Jeep Dodge,"9101 Camp Bowie West Fort Worth, TX 76116",NaN,NaN
4,"$42,560","$42,560",Contact the dealer for pricing details,"Richardson, TX\n28 mi away",0 miles,8-Speed Automatic,Sting-Gray,Black,22 MPG City\n24 MPG Highway\n22 MPG Combined,270 hp 2L I4,Four-Wheel Drive,Gasoline,1C4GJXAN4NW186077,186077,"$42,560",Wrangler,Richardson Chrysler Jeep Dodge Ram,"1321 N Central Expy Richardson, TX 75080",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,"$51,440","$51,440",Contact the dealer for pricing details,"Dallas, TX\n18 mi away",10 miles,8-Speed Automatic,Silver Zynith Clearcoat,Black,22 MPG City\n29 MPG Highway\n25 MPG Combined,260 hp 3L V6 Diesel,Four-Wheel Drive,Diesel,1C4HJXDM1NW242810,F0673,"$51,440",Wrangler,Forest Lane Chrysler Dodge Jeep Ram,"2536 Forest Ln Dallas, TX 75234",NaN,NaN
118,"$51,440","$51,440",Contact the dealer for pricing details,"Dallas, TX\n18 mi away",10 miles,8-Speed Automatic,Silver Zynith Clearcoat,Black,22 MPG City\n29 MPG Highway\n25 MPG Combined,260 hp 3L V6 Diesel,Four-Wheel Drive,Diesel,1C4HJXDM1NW226641,F0743,"$51,440",Wrangler,Forest Lane Chrysler Dodge Jeep Ram,"2536 Forest Ln Dallas, TX 75234",NaN,NaN
119,"$51,440","$51,440",Contact the dealer for pricing details,"Dallas, TX\n18 mi away",---,8-Speed Automatic,Black Clearcoat,Black,22 MPG City\n29 MPG Highway\n25 MPG Combined,260 hp 3L V6 Diesel,Four-Wheel Drive,Diesel,1C4HJXDMXNW226640,F0702,"$51,440",Wrangler,Forest Lane Chrysler Dodge Jeep Ram,"2536 Forest Ln Dallas, TX 75234",NaN,NaN
120,"$51,440","$51,440",Contact the dealer for pricing details,"Dallas, TX\n18 mi away",---,8-Speed Automatic,Silver Zynith Clearcoat,Black,22 MPG City\n29 MPG Highway\n25 MPG Combined,260 hp 3L V6 Diesel,Four-Wheel Drive,Diesel,1C4HJXDM5NW242809,F0686,"$51,440",Wrangler,Forest Lane Chrysler Dodge Jeep Ram,"2536 Forest Ln Dallas, TX 75234",NaN,NaN


In [9]:
len(failed_tries)

39

In [73]:
search_str = "IpF2YF"
title = browser.find_by_tag('h1')
title[0].html.split(' ')[-1]

'$59,565'